# <span style='color:SteelBlue'>P7 - Implémentez un modèle  de scoring (part 4)</span>

![Logo pret_a_depenser](https://user.oc-static.com/upload/2023/03/22/16794938722698_Data%20Scientist-P7-01-banner.png)

# <span style='background:white; color:black'>Sommaire</span>

**Introduction**

Importations des librairies<br>

Chemins d'accès<br>

Fonctions <br>

**Fonction principale**

**Conclusion**

# <span style='background:blue'>Introduction</span>

L'entreprise **"Prêt à dépenser"** souhaite mettre en œuvre un outil de “scoring crédit” pour calculer la probabilité qu’un client rembourse son crédit, puis classifie la demande en crédit accordé ou refusé. Elle souhaite donc développer un **algorithme de classification** en s’appuyant sur des sources de données variées (données comportementales, données provenant d'autres institutions financières, etc.).
___
**Missions**<br>
1/ Construire un modèle de scoring qui donnera une prédiction sur la probabilité de faillite d'un client de façon automatique.

2/ Analyser les features qui contribuent le plus au modèle, d’une manière générale (feature importance globale) et au niveau d’un client (feature importance locale), afin, dans un soucis de transparence, de permettre à un chargé d’études de mieux comprendre le score attribué par le modèle.

3/ Mettre en production le modèle de scoring de prédiction à l’aide d’une API et réaliser une interface de test de cette API.

4/ Mettre en oeuvre une approche globale MLOps de bout en bout, du tracking des expérimentations à l’analyse en production du data drift.
___
Ce notebook presente la 4ème partie concernant l'analyse du data drift.

# <span style='background:grey'>Importations des librairies utilisees dans ce notebook</span>

In [ ]:
import os
import pandas as pd
import numpy as np
import time
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset
from evidently import ColumnMapping

# <span style='background:grey'>Chemins d'accès</span>

In [ ]:
# Chemin du répertoire racine
ROOT_DIR = "C:\\Users\\pierr\\VSC_Projects\\Projet7_OCR_DataScientist"

# Chemin du fichiers des données de reference (application_train)
DATA_REF_PATH = os.path.join(
    ROOT_DIR, "data", "cleaned", "application_train_cleaned.csv"
)

# Chemin du fichier des données nouvelles (application_test)
DATA_NEW_PATH = os.path.join(
    ROOT_DIR, "data", "cleaned", "application_test_cleaned.csv"
)

# <span style='background:grey'>Fonctions</span>

In [ ]:
def load_data(path, drop_columns):
    """
    Charge les données à partir d'un fichier CSV et supprime les colonnes
    spécifiées.
    """
    data = pd.read_csv(path)
    data.drop(columns=drop_columns, inplace=True)
    return data

In [ ]:
def get_categorical_columns(data):
    """
    Récupère les colonnes catégorielles d'un DataFrame (col avec des valeurs
    uniques de 0, 1 et NaN).
    """
    categorical_col = [
        col for col in data.columns
        if set(data[col].unique()).issubset({0, 1, np.nan})
    ]
    return categorical_col

In [ ]:
def get_numerical_columns(data, categorical_col):
    """
    Récupère les colonnes numériques d'un DataFrame (toutes les colonnes sauf
    les colonnes catégorielles).
    """
    all_columns = set(data.columns)
    numerical_columns = list(all_columns - set(categorical_col))
    return numerical_columns

In [ ]:
def create_data_drift_report(
    numerical_columns,
    categorical_col,
    threshold=0.05
):
    """
    Crée un rapport de data drift avec les colonnes numériques et 
    catégorielles.
    """
    # Création du column mapping (selon cat et num)
    column_mapping = ColumnMapping()
    column_mapping.numerical_features = numerical_columns
    column_mapping.categorical_features = categorical_col

    # Création du rapport de data drift
    data_drift_rapport = Report(
        metrics=[DataDriftPreset(
            num_stattest='ks',
            cat_stattest='psi',
            num_stattest_threshold=threshold,
            cat_stattest_threshold=threshold
        )]
    )
    return data_drift_rapport, column_mapping

# <span style='background:blue'>Fonction principale</span>

In [ ]:
def main():
    """
    Fonction principale pour comparer les données de référence et les données
    actuelles (analyse du data drift).
    """
    # Chargement des données
    drop_columns = ["Unnamed: 0", "TARGET", "SK_ID_CURR"]
    data_ref = load_data(DATA_REF_PATH, drop_columns)
    data_new = load_data(DATA_NEW_PATH, drop_columns)

    # Isolement des noms des colonnes numériques et catégorielles
    categorical_col = get_categorical_columns(data_ref)
    numerical_columns = get_numerical_columns(data_ref, categorical_col)

    # Vérification de la similarité des colonnes des 2 DataFrames
    assert set(data_ref.columns) == set(data_new.columns)

    # Création du rapport de data drift
    start_time = time.time()
    data_drift_rapport, column_mapping = create_data_drift_report(
        numerical_columns,
        categorical_col,
        threshold=0.05
    )

    # Run du rapport
    start_time = time.time()
    data_drift_rapport.run(
        reference_data=data_ref,
        current_data=data_new,
        column_mapping=column_mapping
    )

    # Sauvegarde du rapport
    data_drift_rapport.save_html(os.path.join(
        ROOT_DIR,
        "Berthe_Pierrick_5_Tableau_HTML_data_drift_evidently_022024.html"
    ))

# =========== étape 5 : Exécution de la fonction principale ===============
if __name__ == "__main__":
    main()

Le rapport de data drift est stocké dans le fichier **Berthe_Pierrick_5_Tableau_HTML_data_drift_evidently_022024.html**

# <span style='background:blue'>Conclusions</span>

Le rapport indique que 99 des 348 colonnes des données nouvelles (application_test) soit 28,4% des colonnes présentent un data drift. Cela signifie qu'environ un quart des colonnes ont des distributions différentes entre les données d'entraînement et les données de test. Le reste des colonnes, soit les trois quart des colonnes affichent des distributions similaires. 

Nous pouvons donc conclure que le nouveau dataset ne présente pas de data drift puisque le seuil pour un data drift est qu'au moins la moitié (>50%) des colonnes du nouveau dataset doivent présenter une distribution différente de l'ancien dataset pour qu'un data drift soit constaté. 